<a href="https://colab.research.google.com/github/fegvebhtrbtr/ColabFold/blob/main/AlphaFold2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/sokrypton/ColabFold/main/.github/ColabFold_Marv_Logo_Small.png" height="200" align="right" style="height:240px">

## ColabFold v1.5.5: 使用MMseqs2的AlphaFold2

易于使用的蛋白质结构和复合物预测工具，使用 [AlphaFold2](https://www.nature.com/articles/s41586-021-03819-2) 和 [Alphafold2-multimer](https://www.biorxiv.org/content/10.1101/2021.10.04.463034v1)。序列比对/模板通过 [MMseqs2](mmseqs.com) 和 [HHsearch](https://github.com/soedinglab/hh-suite) 生成。更多详情请参见笔记本<a href="#说明">底部</a>，查看 [ColabFold GitHub](https://github.com/sokrypton/ColabFold) 和 [Nature Protocols](https://www.nature.com/articles/s41596-024-01060-5)。

旧版本: [v1.4](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.4.0/AlphaFold2.ipynb), [v1.5.1](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.5.1/AlphaFold2.ipynb), [v1.5.2](https://colab.research.google.com/github/sokrypton/ColabFold/blob/v1.5.2/AlphaFold2.ipynb), [v1.5.3-patch](https://colab.research.google.com/github/sokrypton/ColabFold/blob/56c72044c7d51a311ca99b953a71e552fdc042e1/AlphaFold2.ipynb)

[Mirdita M, Schütze K, Moriwaki Y, Heo L, Ovchinnikov S, Steinegger M. ColabFold: Making protein folding accessible to all.
*Nature Methods*, 2022](https://www.nature.com/articles/s41592-022-01488-1)

In [ ]:
#@title 输入蛋白质序列，然后点击 `运行时` -> `全部运行`
from google.colab import files
import os
import re
import hashlib
import random

from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

query_sequence = 'PIAQIHILEGRSDEQKETLIREVSEAISRSLDAPLTSVRVIITEMAKGHFGIGGELASK' #@param {type:"string"}
#@markdown  - 使用 `:` 指定蛋白质间的链断裂以**建模复合物**（支持同源和异源寡聚体）。例如，**PI...SK:PI...SK** 表示同源二聚体
jobname = 'test' #@param {type:"string"}
# 要使用的模型数量
num_relax = 0 #@param [0, 1, 5] {type:"raw"}
#@markdown - 指定使用amber对多少个顶级结构进行松弛处理
template_mode = "none" #@param ["none", "pdb100","custom"]
#@markdown - `none` = 不使用模板信息。 `pdb100` = 在pdb100中检测模板（参见[说明](#pdb100)）。 `custom` - 上传并搜索自己的模板（PDB或mmCIF格式，参见[说明](#custom_templates)）

use_amber = num_relax > 0

# 移除空格
query_sequence = "".join(query_sequence.split())

basejobname = "".join(jobname.split())
basejobname = re.sub(r'\W+', '', basejobname)
jobname = add_hash(basejobname, query_sequence)

# 检查是否存在具有jobname的目录
def check(folder):
  if os.path.exists(folder):
    return False
  else:
    return True
if not check(jobname):
  n = 0
  while not check(f"{jobname}_{n}"): n += 1
  jobname = f"{jobname}_{n}"

# 创建目录保存结果
os.makedirs(jobname, exist_ok=True)

# 保存查询
queries_path = os.path.join(jobname, f"{jobname}.csv")
with open(queries_path, "w") as text_file:
  text_file.write(f"id,sequence\n{jobname},{query_sequence}")

if template_mode == "pdb100":
  use_templates = True
  custom_template_path = None
elif template_mode == "custom":
  custom_template_path = os.path.join(jobname,f"template")
  os.makedirs(custom_template_path, exist_ok=True)
  uploaded = files.upload()
  use_templates = True
  for fn in uploaded.keys():
    os.rename(fn,os.path.join(custom_template_path,fn))
else:
  custom_template_path = None
  use_templates = False

print("任务名称",jobname)
print("序列",query_sequence)
print("长度",len(query_sequence.replace(":","")))

In [ ]:
#@title 安装依赖
%%time
import os
USE_AMBER = use_amber
USE_TEMPLATES = use_templates
PYTHON_VERSION = python_version

if not os.path.isfile("COLABFOLD_READY"):
  print("正在安装colabfold...")
  os.system("pip install -q --no-warn-conflicts 'colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold'")
  if os.environ.get('TPU_NAME', False) != False:
    os.system("pip uninstall -y jax jaxlib")
    os.system("pip install --no-warn-conflicts --upgrade dm-haiku==0.0.10 'jax[cuda12_pip]'==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold")
  # 修复TF崩溃的hack
  os.system("rm -f /usr/local/lib/python3.*/dist-packages/tensorflow/core/kernels/libtfkernel_sobol_op.so")
  os.system("touch COLABFOLD_READY")

if USE_AMBER or USE_TEMPLATES:
  if not os.path.isfile("CONDA_READY"):
    print("正在安装conda...")
    os.system("wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh")
    os.system("bash Miniforge3-Linux-x86_64.sh -bfp /usr/local")
    os.system("mamba config --set auto_update_conda false")
    os.system("touch CONDA_READY")

if USE_TEMPLATES and not os.path.isfile("HH_READY") and USE_AMBER and not os.path.isfile("AMBER_READY"):
  print("正在安装hhsuite和amber...")
  os.system(f"mamba install -y -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 openmm=8.2.0 python='{PYTHON_VERSION}' pdbfixer")
  os.system("touch HH_READY")
  os.system("touch AMBER_READY")
else:
  if USE_TEMPLATES and not os.path.isfile("HH_READY"):
    print("正在安装hhsuite...")
    os.system(f"mamba install -y -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python='{PYTHON_VERSION}'")
    os.system("touch HH_READY")
  if USE_AMBER and not os.path.isfile("AMBER_READY"):
    print("正在安装amber...")
    os.system(f"mamba install -y -c conda-forge openmm=8.2.0 python='{PYTHON_VERSION}' pdbfixer")
    os.system("touch AMBER_READY")

In [ ]:
#@markdown ### MSA选项（自定义MSA上传、单序列、配对模式）
msa_mode = "mmseqs2_uniref_env" #@param ["mmseqs2_uniref_env", "mmseqs2_uniref","single_sequence","custom"]
pair_mode = "unpaired_paired" #@param ["unpaired_paired","paired","unpaired"] {type:"string"}
#@markdown - "unpaired_paired" = 来自同一物种的配对序列 + 未配对的MSA, "unpaired" = 每条链单独的MSA, "paired" - 仅使用配对序列

# 决定使用哪个a3m文件
if "mmseqs2" in msa_mode:
  a3m_file = os.path.join(jobname,f"{jobname}.a3m")

elif msa_mode == "custom":
  a3m_file = os.path.join(jobname,f"{jobname}.custom.a3m")
  if not os.path.isfile(a3m_file):
    custom_msa_dict = files.upload()
    custom_msa = list(custom_msa_dict.keys())[0]
    header = 0
    import fileinput
    for line in fileinput.FileInput(custom_msa,inplace=1):
      if line.startswith(">"):
         header = header + 1
      if not line.rstrip():
        continue
      if line.startswith(">") == False and header == 1:
         query_sequence = line.rstrip()
      print(line, end='')

    os.rename(custom_msa, a3m_file)
    queries_path=a3m_file
    print(f"移动 {custom_msa} 到 {a3m_file}")

else:
  a3m_file = os.path.join(jobname,f"{jobname}.single_sequence.a3m")
  with open(a3m_file, "w") as text_file:
    text_file.write(">1\n%s" % query_sequence)

In [ ]:
#@markdown ### 高级设置
model_type = "auto" #@param ["auto", "alphafold2_ptm", "alphafold2_multimer_v1", "alphafold2_multimer_v2", "alphafold2_multimer_v3", "deepfold_v1", "alphafold2"]
#@markdown - 如果选择 `auto`，将对单体预测使用 `alphafold2_ptm`，对复合物预测使用 `alphafold2_multimer_v3`。
#@markdown 可以使用任何模型类型（无论输入是单体还是复合物）。
num_recycles = "3" #@param ["auto", "0", "1", "3", "6", "12", "24", "48"]
#@markdown - 如果选择 `auto`，当 `model_type=alphafold2_multimer_v3` 时使用 `num_recycles=20`，否则使用 `num_recycles=3`。
recycle_early_stop_tolerance = "auto" #@param ["auto", "0.0", "0.5", "1.0"]
#@markdown - 如果选择 `auto`，当 `model_type=alphafold2_multimer_v3` 时使用 `tol=0.5`，否则使用 `tol=0.0`。
relax_max_iterations = 200 #@param [0, 200, 2000] {type:"raw"}
#@markdown - 最大amber松弛迭代次数，`0` = 无限制（AlphaFold2默认，可能耗时很长）
pairing_strategy = "greedy" #@param ["greedy", "complete"] {type:"string"}
#@markdown - `greedy` = 配对任何分类学匹配的子集，`complete` = 所有序列必须在一行中匹配
calc_extra_ptm = False #@param {type:"boolean"}
#@markdown - 返回链间iptm/actifptm

#@markdown #### 采样设置
#@markdown - 启用dropout并增加种子数以从模型的不确定性中采样预测。
#@markdown - 减少 `max_msa` 以增加不确定性
max_msa = "auto" #@param ["auto", "512:1024", "256:512", "64:128", "32:64", "16:32"]
num_seeds = 1 #@param [1,2,4,8,16] {type:"raw"}
use_dropout = False #@param {type:"boolean"}

num_recycles = None if num_recycles == "auto" else int(num_recycles)
recycle_early_stop_tolerance = None if recycle_early_stop_tolerance == "auto" else float(recycle_early_stop_tolerance)
if max_msa == "auto": max_msa = None

#@markdown #### 保存设置
save_all = False #@param {type:"boolean"}
save_recycles = False #@param {type:"boolean"}
save_to_google_drive = False #@param {type:"boolean"}
#@markdown - 如果选择了save_to_google_drive选项，结果zip文件将上传到您的Google Drive
dpi = 200 #@param {type:"integer"}
#@markdown - 设置图像分辨率dpi

if save_to_google_drive:
  from pydrive2.drive import GoogleDrive
  from pydrive2.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("您已登录Google Drive，可以开始使用了！")

#@markdown 更新表单后不要忘记点击 `运行时` -> `全部运行`。

In [ ]:
#@title 运行预测
display_images = True #@param {type:"boolean"}

import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from Bio import BiopythonDeprecationWarning
warnings.simplefilter(action='ignore', category=BiopythonDeprecationWarning)
from pathlib import Path
from colabfold.download import download_alphafold_params, default_data_dir
from colabfold.utils import setup_logging
from colabfold.batch import get_queries, run, set_model_type
from colabfold.plot import plot_msa_v2

import os
import numpy as np
try:
  K80_chk = os.popen('nvidia-smi | grep "Tesla K80" | wc -l').read()
except:
  K80_chk = "0"
  pass
if "1" in K80_chk:
  print("警告：检测到GPU Tesla K80：总长度限制 < 1000")
  if "TF_FORCE_UNIFIED_MEMORY" in os.environ:
    del os.environ["TF_FORCE_UNIFIED_MEMORY"]
  if "XLA_PYTHON_CLIENT_MEM_FRACTION" in os.environ:
    del os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]

from colabfold.colabfold import plot_protein
from pathlib import Path
import matplotlib.pyplot as plt

# 为了导入pdbfixer需要这个
if use_amber and f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
    sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

def input_features_callback(input_features):
  if display_images:
    plot_msa_v2(input_features)
    plt.show()
    plt.close()

def prediction_callback(protein_obj, length,
                        prediction_result, input_features, mode):
  model_name, relaxed = mode
  if not relaxed:
    if display_images:
      fig = plot_protein(protein_obj, Ls=length, dpi=150)
      plt.show()
      plt.close()

result_dir = jobname
log_filename = os.path.join(jobname,"log.txt")
setup_logging(Path(log_filename))

queries, is_complex = get_queries(queries_path)
model_type = set_model_type(is_complex, model_type)

if "multimer" in model_type and max_msa is not None:
  use_cluster_profile = False
else:
  use_cluster_profile = True

download_alphafold_params(model_type, Path("."))
results = run(
    queries=queries,
    result_dir=result_dir,
    use_templates=use_templates,
    custom_template_path=custom_template_path,
    num_relax=num_relax,
    msa_mode=msa_mode,
    model_type=model_type,
    num_models=5,
    num_recycles=num_recycles,
    relax_max_iterations=relax_max_iterations,
    recycle_early_stop_tolerance=recycle_early_stop_tolerance,
    num_seeds=num_seeds,
    use_dropout=use_dropout,
    model_order=[1,2,3,4,5],
    is_complex=is_complex,
    data_dir=Path("."),
    keep_existing_results=False,
    rank_by="auto",
    pair_mode=pair_mode,
    pairing_strategy=pairing_strategy,
    stop_at_score=float(100),
    prediction_callback=prediction_callback,
    dpi=dpi,
    zip_results=False,
    save_all=save_all,
    max_msa=max_msa,
    use_cluster_profile=use_cluster_profile,
    input_features_callback=input_features_callback,
    save_recycles=save_recycles,
    user_agent="colabfold/google-colab-main",
    calc_extra_ptm=calc_extra_ptm,
)
results_zip = f"{jobname}.result.zip"
os.system(f"zip -r {results_zip} {jobname}")

In [ ]:
#@title 显示3D结构 {run: "auto"}
import py3Dmol
import glob
import matplotlib.pyplot as plt
from colabfold.colabfold import plot_plddt_legend
from colabfold.colabfold import pymol_color_list, alphabet_list
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

tag = results["rank"][0][rank_num - 1]
jobname_prefix = ".custom" if msa_mode == "custom" else ""
pdb_filename = f"{jobname}/{jobname}{jobname_prefix}_unrelaxed_{tag}.pdb"
pdb_file = glob.glob(pdb_filename)

def show_pdb(rank_num=1, show_sidechains=False, show_mainchains=False, color="lDDT"):
  model_name = f"rank_{rank_num}"
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_file[0],'r').read(),'pdb')

  if color == "lDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    chains = len(queries[0][1]) + 1 if is_complex else 1
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})

  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()
  return view

show_pdb(rank_num, show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show()

In [ ]:
#@title 图表 {run: "auto"}
from IPython.display import display, HTML
import base64
from html import escape

# 参见: https://stackoverflow.com/a/53688522
def image_to_data_url(filename):
  ext = filename.split('.')[-1]
  prefix = f'data:image/{ext};base64,'
  with open(filename, 'rb') as f:
    img = f.read()
  return prefix + base64.b64encode(img).decode('utf-8')

pae = ""
pae_file = os.path.join(jobname,f"{jobname}{jobname_prefix}_pae.png")
if os.path.isfile(pae_file):
    pae = image_to_data_url(pae_file)
cov = image_to_data_url(os.path.join(jobname,f"{jobname}{jobname_prefix}_coverage.png"))
plddt = image_to_data_url(os.path.join(jobname,f"{jobname}{jobname_prefix}_plddt.png"))
display(HTML(f"""
<style>
  img {{
    float:left;
  }}
  .full {{
    max-width:100%;
  }}
  .half {{
    max-width:50%;
  }}
  @media (max-width:640px) {{
    .half {{
      max-width:100%;
    }}
  }}
</style>
<div style="max-width:90%; padding:2em;">
  <h1>{escape(jobname)}的图表</h1>
  { '<!--' if pae == '' else '' }<img src="{pae}" class="full" />{ '-->' if pae == '' else '' }
  <img src="{cov}" class="half" />
  <img src="{plddt}" class="half" />
</div>
"""))

In [ ]:
#@title 打包并下载结果
#@markdown 如果在下载结果存档时遇到问题，请尝试禁用广告拦截器并再次运行此单元格。如果仍然失败，请点击左侧的小文件夹图标，导航到文件：`jobname.result.zip`，右键单击并选择"下载"（参见[截图](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)）。

if msa_mode == "custom":
  print("不要忘记引用您的自定义MSA生成方法。")

files.download(f"{jobname}.result.zip")

if save_to_google_drive == True and drive:
  uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
  uploaded.SetContentFile(f"{jobname}.result.zip")
  uploaded.Upload()
  print(f"已将 {jobname}.result.zip 上传到Google Drive，ID为 {uploaded.get('id')}")

# 说明 <a name="说明"></a>
有关详细说明、提示和技巧，请参阅最近在 [Nature Protocols](https://www.nature.com/articles/s41596-024-01060-5) 上发表的文章。

**快速开始**
1. 在输入字段中粘贴您的蛋白质序列。
2. 点击"运行时" -> "全部运行"。
3. 流程包括5个步骤。当前运行的步骤由旁边的带有停止标志的圆圈指示。

**结果zip文件内容**

1. PDB格式的结构，按平均pLDDT排序，复合物按pTMscore排序。（如果启用了`use_amber`，则包含未松弛和松弛的结构）。
2. 模型质量图表。
3. MSA覆盖度图表。
4. 参数日志文件。
5. A3M格式的输入MSA。
6. 使用 [AlphaFold-DB格式](https://alphafold.ebi.ac.uk/faq#faq-7) 的 `predicted_aligned_error_v1.json` 和每个模型的 `scores.json`，其中包含PAE的数组（列表的列表）、平均pLDDT列表和pTMscore。
7. 包含所有使用工具和数据库引用的BibTeX文件。

任务结束时，将弹出下载模态框，其中包含 `jobname.result.zip` 文件。此外，如果选择了 `save_to_google_drive` 选项，`jobname.result.zip` 将上传到您的Google Drive。

**复合物的MSA生成**

对于复合物预测，我们使用未配对和配对的MSA。未配对的MSA与蛋白质结构预测的方式相同，通过搜索UniRef100和环境序列各三次迭代生成。

配对的MSA是通过搜索UniRef100数据库并配对共享相同NCBI分类标识符（=物种或亚种）的最佳命中生成的。只有当所有查询序列都存在于相应的分类标识符中时，我们才配对序列。

**使用自定义MSA作为输入**

要使用自定义MSA（A3M格式）预测结构：（1）将 `msa_mode` 更改为 "custom"，（2）等待上传框出现在"MSA选项..."框的末尾。上传您的A3M文件。A3M的第一个fasta条目必须是没有空格的查询序列。

也可以为复合物预测提供自定义MSA。有关格式的更多信息请阅读[这里](https://github.com/sokrypton/ColabFold/issues/76)。

作为MSA生成的替代方案，可以使用 [HHblits Toolkit服务器](https://toolkit.tuebingen.mpg.de/tools/hhblits)。提交查询后，点击"Query Template MSA" -> "Download Full A3M"。下载A3M文件并在此笔记本中上传。

**PDB100** <a name="pdb100"></a>

自23/06/08起，我们已从使用PDB70过渡到100%聚类的PDB，即PDB100。PDB100的构建方法与PDB70不同。

PDB70是通过将每个PDB70代表性序列通过 [HHblits](https://github.com/soedinglab/hh-suite) 对 [Uniclust30](https://uniclust.mmseqs.com/) 运行构建的。另一方面，PDB100是通过使用 [Foldseek](https://github.com/steineggerlab/foldseek) 搜索每个PDB100代表性结构对 [AlphaFold数据库](https://alphafold.ebi.ac.uk) 构建的。

为了与旧版笔记本和本地安装兼容，生成的文件和API响应将继续命名为"PDB70"，即使我们现在使用的是PDB100。

**使用自定义模板** <a name="custom_templates"></a>

要使用自定义模板（PDB或mmCIF格式）预测结构：（1）在执行单元格中将 `template_mode` 更改为 "custom"，（2）等待上传框出现在"输入蛋白质"框的末尾。选择并上传您的模板（可以选择多个）。

* 模板必须遵循四个字母的PDB命名，使用小写字母。

* mmCIF格式的模板必须包含 `_entity_poly_seq`。如果此字段不存在，则会抛出错误。如果 `_pdbx_audit_revision_history.revision_date` 不存在，则会自动生成。

* PDB格式的模板会自动转换为mmCIF格式。`_entity_poly_seq` 和 `_pdbx_audit_revision_history.revision_date` 会自动生成。

如果遇到问题，请向此[问题](https://github.com/sokrypton/ColabFold/issues/177)报告。

**与完整AlphaFold2和AlphaFold2 Colab的比较**

此笔记本使用MMseqs2替代AlphaFold2的同源性检测和MSA配对。有关与 [AlphaFold2 Colab](https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb) 和完整 [AlphaFold2](https://github.com/deepmind/alphafold) 系统的比较，请阅读我们的[论文](https://www.nature.com/articles/s41592-022-01488-1)。

**故障排除**
* 检查运行时类型是否设置为GPU，在"运行时" -> "更改运行时类型"中。
* 尝试重启会话"运行时" -> "工厂重置运行时"。
* 检查您的输入序列。

**已知问题**
* Google Colab分配不同类型的内存量不同的GPU。有些可能没有足够的内存来预测长序列的结构。
* 您的浏览器可能会阻止弹出下载结果文件。您可以选择 `save_to_google_drive` 选项上传到Google Drive，或者手动下载结果文件：点击左侧的小文件夹图标，导航到文件：`jobname.result.zip`，右键单击并选择"下载"（参见[截图](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)）。

**限制**
* 计算资源：我们的MMseqs2 API每天可以处理约20-50k个请求。
* MSA：MMseqs2非常精确和敏感，但与针对BFD或MGnify搜索的HHblits/HMMer相比，可能找到的命中较少。
* 我们建议另外使用完整的 [AlphaFold2流程](https://github.com/deepmind/alphafold)。

**图表说明**
*   **每个位置的序列数** - 我们希望每个位置至少有30个序列，为了最佳性能，理想情况下为100个序列。
*   **每个位置的预测lDDT** - 每个位置的模型置信度（满分100）。越高越好。
*   **预测对齐误差** - 对于同源寡聚体，这可能是评估模型对界面置信度的有用指标。越低越好。

**错误报告**
- 如果遇到任何错误，请将问题报告至 https://github.com/sokrypton/ColabFold/issues

**许可证**

ColabFold的源代码根据 [MIT](https://raw.githubusercontent.com/sokrypton/ColabFold/main/LICENSE) 许可。此外，此笔记本使用AlphaFold2源代码及其参数，分别根据 [Apache 2.0](https://raw.githubusercontent.com/deepmind/alphafold/main/LICENSE) 和 [CC BY 4.0](https://creativecommons.org/licenses/by-sa/4.0/) 许可。有关AlphaFold许可证的更多信息请阅读[这里](https://github.com/deepmind/alphafold)。

**致谢**
- 我们感谢AlphaFold团队开发了优秀的模型并开源了软件。

- [KOBIC](https://kobic.re.kr) 和 [Söding Lab](https://www.mpinat.mpg.de/soeding) 为MMseqs2 MSA服务器提供计算资源。

- Richard Evans 帮助基准测试ColabFold的Alphafold-multimer支持。

- [David Koes](https://github.com/dkoes) 为他出色的 [py3Dmol](https://3dmol.csb.pitt.edu/) 插件，没有它这些笔记本将会相当无聊！

- Do-Yoon Kim 创建了ColabFold徽标。

- 由 Sergey Ovchinnikov ([@sokrypton](https://twitter.com/sokrypton))、Milot Mirdita ([@milot_mirdita](https://twitter.com/milot_mirdita)) 和 Martin Steinegger ([@thesteinegger](https://twitter.com/thesteinegger)) 制作的colab。
